In [1]:
from triagerx.model.module_factory import ModelFactory
from loguru import logger
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
from triagerx.dataset.text_processor import TextProcessor
from tqdm import tqdm

tqdm.pandas()

In [2]:
torch.manual_seed(42)

In [3]:
dataset_path = "/home/mdafifal.mamun/notebooks/triagerX/data/openj9_dataset_6_7_24_last_contrib.csv"

In [4]:
df = pd.read_csv(dataset_path)

In [5]:
df.head()

,issue_number,issue_title,issue_body,issue_url,issue_state,creator,labels,assignees,component
0,2,Build instructions link in the README.md point...,The `Build instructions` link in the `README.m...,https://github.com/eclipse-openj9/openj9/issues/2,closed,aarongraham9,NaN,gireeshpunathil,NaN
1,3,FAQ link in the README is broken,FAQ link in the README leads to: http://www.ec...,https://github.com/eclipse-openj9/openj9/issues/3,closed,dorrab,NaN,mpirvu,NaN
2,5,Link to DockerFile on build instruction page i...,Link for DockerFile on [build instruction page...,https://github.com/eclipse-openj9/openj9/issues/5,closed,r30shah,NaN,r30shah,NaN
3,11,HOWTO Request: Managing changes across depende...,"Like all projects, OpenJ9 builds on the should...",https://github.com/eclipse-openj9/openj9/issue...,open,mgaudet,question,hzongaro,NaN
4,13,Compilation Output is too Verbose,The output when compiling the OpenJ9 source co...,https://github.com/eclipse-openj9/openj9/issue...,closed,rservant,"enhancement, comp:build",hzongaro,comp:build


In [6]:
df = df.rename(columns={"assignees": "owner", "issue_body": "description"})
df = df.sort_values(by="issue_number")
df = df[df["owner"].notna()]
# df = df[df["component"].notna()]

In [7]:
target_components = ["comp:vm", "comp:jvmti", "comp:jclextensions", "comp:test", "comp:build", "comp:gc"]
target_components = sorted(target_components)

In [8]:
# Define active user map
vm_users = [
    "pshipton",
    "keithc-ca",
    "gacholio",
    "tajila",
    "babsingh",
    "JasonFengJ9",
    "fengxue-IS",
    "hangshao0",
    "theresa.t.mammarella",
    "ChengJin01",
    "singh264",
    "thallium",
    "ThanHenderson",
]
jvmti_users = ["gacholio", "tajila", "babsingh", "fengxue-IS"]
jclextensions_users = ["JasonFengJ9", "pshipton", "keithc-ca"]
test_users = ["LongyuZhang", "annaibm", "sophiaxu0424", "KapilPowar", "llxia"]
build_users = ["adambrousseau", "mahdipub"]
gc_users = ["dmitripivkine", "amicic", "kangyining", "LinHu2016"]

# Putting them in dictionaries
components = {
    "comp:vm": vm_users,
    "comp:jvmti": jvmti_users,
    "comp:jclextensions": jclextensions_users,
    "comp:test": test_users,
    "comp:build": build_users,
    "comp:gc": gc_users,
}

expected_users = [user for user_list in components.values() for user in user_list]
df = df[df["owner"].isin(expected_users)]

# df = df[df["component"].isin(target_components)]

In [9]:
df = df.sort_values(by="issue_number")
df["text"] = df.apply(
        lambda x: "Bug Title: " + str(x["issue_title"]) + "\nBug Description: " + str(x["description"]),
        axis=1,
    )

df.head()

,issue_number,issue_title,description,issue_url,issue_state,creator,labels,owner,component,text
8,26,Solve the 'inline' consternation,The [definition of JITINLINE](https://github.c...,https://github.com/eclipse-openj9/openj9/issue...,open,mgaudet,comp:jit,tajila,comp:jit,Bug Title: Solve the 'inline' consternation\nB...
10,30,cpu entitlement when hypervisor is present,I believe there are two problems in JIT with r...,https://github.com/eclipse-openj9/openj9/issue...,closed,ashu-mehra,"bug, comp:jit",tajila,comp:jit,Bug Title: cpu entitlement when hypervisor is ...
11,32,StackOverflow #OpenJ9 tag doesn't exist,The [OpenJ9 landing page](https://www.eclipse....,https://github.com/eclipse-openj9/openj9/issue...,closed,kgibm,NaN,tajila,NaN,Bug Title: StackOverflow #OpenJ9 tag doesn't e...
12,33,document tr.jit options,It would be worthwhile to document the hidden ...,https://github.com/eclipse-openj9/openj9/issue...,closed,gireeshpunathil,comp:doc,tajila,comp:doc,Bug Title: document tr.jit options\nBug Descri...
13,36,Supporting OS X Mac builds,It has been pointed out that Eclipse OpenJ9 do...,https://github.com/eclipse-openj9/openj9/issue...,closed,mstoodle,"enhancement, help wanted, doc:externals, userR...",tajila,NaN,Bug Title: Supporting OS X Mac builds\nBug Des...


In [10]:
df_train, df_test = train_test_split(df, test_size=0.1, shuffle=False)

In [11]:
sample_threshold = 20
developers = df_train["owner"].value_counts()
filtered_developers = developers.index[developers >= sample_threshold]
df_train = df_train[df_train["owner"].isin(filtered_developers)]

train_owners = set(df_train["owner"])
test_owners = set(df_test["owner"])

unwanted = list(test_owners - train_owners)

df_test = df_test[~df_test["owner"].isin(unwanted)]

logger.info(f"Training data: {len(df_train)}, Validation data: {len(df_test)}")
logger.info(f"Number of developers: {len(df_train.owner.unique())}")

logger.info(f"Train dataset size: {len(df_train)}")
logger.info(f"Test dataset size: {len(df_test)}")

2024-06-19 11:15:08.329 | INFO     | __main__:<module>:13 - Training data: 1831, Validation data: 184
2024-06-19 11:15:08.331 | INFO     | __main__:<module>:14 - Number of developers: 15
2024-06-19 11:15:08.332 | INFO     | __main__:<module>:16 - Train dataset size: 1831
2024-06-19 11:15:08.333 | INFO     | __main__:<module>:17 - Test dataset size: 184


In [12]:
# # Generate label ids
lbl2idx = {}
idx2lbl = {}

train_owners = sorted(train_owners)

for idx, dev in enumerate(train_owners):
    lbl2idx[dev] = idx
    idx2lbl[idx] = dev

df_train["owner_id"] = df_train["owner"].apply(lambda owner: lbl2idx[owner])
df_test["owner_id"] = df_test["owner"].apply(lambda owner: lbl2idx[owner])

In [13]:
df_train = df_train[df_train["component"].notna()]
df_test = df_test[df_test["component"].notna()]

In [14]:
comp_id2label = {}
comp_lbl2id = {}

for i, comp in enumerate(target_components):
    comp_id2label[i] = comp
    comp_lbl2id[comp] = i

In [15]:
base_transformer_models = ["microsoft/deberta-base", "roberta-base"]

In [16]:
developer_model_weights = "/work/disa_lab/projects/triagerx/models/openj9/triagerx_ensemble_u3_15_classes_last_dev_seed42.pt"
component_model_weights = "/work/disa_lab/projects/triagerx/models/openj9/component_triagerx_u3_6_classes_seed42.pt"

In [17]:
logger.debug("Modeling network...")
dev_model = ModelFactory.get_model(
    model_key="triagerx",
    output_size=len(df_train.owner_id.unique()),
    unfrozen_layers=3,
    num_classifiers=3,
    base_models=base_transformer_models,
    dropout=0.2,
    max_tokens=256,
    label_map=idx2lbl,
)

2024-06-19 11:15:08.750 | DEBUG    | __main__:<module>:1 - Modeling network...
2024-06-19 11:15:08.752 | DEBUG    | triagerx.model.module_factory:get_model:54 - Instantiating model of class: <class 'triagerx.model.triagerx_dev_model.TriagerxDevModel'>
2024-06-19 11:15:08.753 | DEBUG    | triagerx.model.module_factory:get_model:65 - Including number of filters and classifiers
2024-06-19 11:15:08.755 | DEBUG    | triagerx.model.triagerx_dev_model:__init__:24 - Loading base transformer models: ['microsoft/deberta-base', 'roberta-base']
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
dev_model.load_state_dict(torch.load(developer_model_weights))

<All keys matched successfully>

In [19]:
comp_model = ModelFactory.get_model(
    model_key="triagerx",
    output_size=6,
    unfrozen_layers=3,
    num_classifiers=3,
    base_models=["microsoft/deberta-base", "roberta-base"],
    dropout=0.2,
    max_tokens=256,
    label_map=comp_id2label,
)

2024-06-19 11:15:13.149 | DEBUG    | triagerx.model.module_factory:get_model:54 - Instantiating model of class: <class 'triagerx.model.triagerx_dev_model.TriagerxDevModel'>
2024-06-19 11:15:13.151 | DEBUG    | triagerx.model.module_factory:get_model:65 - Including number of filters and classifiers
2024-06-19 11:15:13.152 | DEBUG    | triagerx.model.triagerx_dev_model:__init__:24 - Loading base transformer models: ['microsoft/deberta-base', 'roberta-base']
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
comp_model.load_state_dict(torch.load(component_model_weights))

<All keys matched successfully>

In [21]:
similarity_model = SentenceTransformer("all-mpnet-base-v2")

In [22]:
train_embeddings_path = "/home/mdafifal.mamun/notebooks/triagerX/data/openj9_embeddings/embeddings.npy"

In [23]:
encodings = similarity_model.encode(
    df_train.text.tolist(),
    show_progress_bar=True
)

import numpy as np

np.save(train_embeddings_path, encodings)

Batches:   0%|          | 0/46 [00:00<?, ?it/s]

In [33]:
from datetime import datetime
import json
import os
from collections import defaultdict
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from loguru import logger
from sentence_transformers import util

from triagerx.dataset.text_processor import TextProcessor
from triagerx.model.prediction_model import PredictionModel


class TriagerX:
    """
    A class for recommending developers and components for issues based on prediction models
    and similarity measures.
    """

    def __init__(
        self,
        component_prediction_model: PredictionModel,
        developer_prediction_model: PredictionModel,
        similarity_model: nn.Module,
        train_data: pd.DataFrame,
        train_embeddings: str,
        issues_path: str,
        developer_id_map: Dict[str, int],
        component_id_map: Dict[str, int],
        component_developers_map: Dict[str, List[str]],
        device: str,
    ) -> None:
        """
        Initializes the TriagerX system with models, data, and configurations.

        Args:
            component_prediction_model (PredictionModel): Model to predict components.
            developer_prediction_model (PredictionModel): Model to predict developers.
            similarity_model (nn.Module): Model to compute similarity between issues.
            train_data (pd.DataFrame): Training data containing issue information.
            train_embeddings (str): Path to precomputed embeddings of training issues.
            issues_path (str): Path to the directory containing issue files.
            developer_id_map (Dict[str, int]): Mapping from developer names to IDs.
            component_id_map (Dict[str, int]): Mapping from component names to IDs.
            component_developers_map (Dict[str, List[str]]): Mapping from components to associated developers.
            device (str): Device to run the models on (e.g., 'cpu' or 'cuda').
        """
        self._component_prediction_model = component_prediction_model.to(device)
        self._developer_prediction_model = developer_prediction_model.to(device)
        self._similarity_model = similarity_model
        self._device = device
        self._train_data = train_data
        self._special_tokens = TextProcessor.SPECIAL_TOKENS
        self._issues_path = issues_path
        self._all_issues = os.listdir(issues_path)
        self._developer2id_map = developer_id_map
        self._component2id_map = component_id_map
        self._id2developer_map = {idx: dev for dev, idx in developer_id_map.items()}
        self._id2component_map = {idx: comp for comp, idx in component_id_map.items()}
        self._component_developers_map = component_developers_map
        self._expected_developers = [
            dev for devs in component_developers_map.values() for dev in devs
        ]
        logger.debug(f"Using device: {device}")
        logger.debug("Loading embeddings for existing issues...")
        self._all_embeddings = np.load(train_embeddings)

    def get_recommendation(
        self, issue: str, k_comp: int, k_dev: int, k_rank: int, similarity_threshold: float
    ) -> Dict[str, List]:
        """
        Generates recommendations for developers and components for a given issue.

        Args:
            issue (str): The issue description.
            k_comp (int): Number of top components to predict.
            k_dev (int): Number of top developers to recommend.
            k_rank (int): Number of developers to rank.
            similarity_threshold (float): Similarity threshold to filter similar issues.

        Returns:
            Dict[str, List]: A dictionary with predicted components, developers, and their scores.
        """
        logger.debug("Predicting components...")
        comp_prediction_score, predicted_components = self._get_predicted_components(issue, k_comp)
        predicted_components_name = [self._id2component_map[idx] for idx in predicted_components]
        logger.info(f"Predicted components: {predicted_components_name}")

        logger.debug("Generating developer recommendations...")
        dev_prediction_score, predicted_devs = self._get_recommendation_from_dev_model(issue, k_rank)
        predicted_developers_name = [self._id2developer_map[idx] for idx in predicted_devs]
        logger.info(f"Recommended developers: {predicted_developers_name}")

        logger.debug("Generating recommendations by similarity...")
        dev_predictions_by_similarity = self._get_recommendation_by_similarity(
            issue, predicted_components, k_dev, k_rank, similarity_threshold
        )
        similar_issue_devs = [dev_sim for dev_sim, _ in dev_predictions_by_similarity]
        logger.info(f"Recommended developers by issue similarity: {similar_issue_devs}")

        logger.debug("Aggregating rankings...")
        rank_lists = [predicted_developers_name, similar_issue_devs]
        aggregated_rank = self._aggregate_rank(rank_lists)[:k_dev]
        logger.info(f"Recommended developers by ranking aggregation: {aggregated_rank}")

        recommendations = {
            "predicted_components": predicted_components_name,
            "comp_prediction_score": comp_prediction_score,
            "predicted_developers": predicted_developers_name[:k_dev],
            "dev_prediction_score": dev_prediction_score,
            "similar_devs": dev_predictions_by_similarity,
            "combined_ranking": aggregated_rank,
        }

        return recommendations

    def _aggregate_rank(self, rank_lists: List[List[str]]) -> List[str]:
        """
        Aggregates rankings from different lists using Borda count.

        Args:
            rank_lists (List[List[str]]): Lists of rankings to aggregate.

        Returns:
            List[str]: Aggregated ranking.
        """
        borda_scores = defaultdict(int)
        for rank_list in rank_lists:
            for i, item in enumerate(rank_list):
                borda_scores[item] += len(rank_list) - i
        sorted_items = sorted(borda_scores.items(), key=lambda x: x[1], reverse=True)
        return [item[0] for item in sorted_items]

    def _get_recommendation_from_dev_model(self, issue: str, k: int) -> Tuple[List[float], List[int]]:
        """
        Generates developer recommendations using the developer prediction model.

        Args:
            issue (str): The issue description.
            k (int): Number of top developers to recommend.

        Returns:
            Tuple[List[float], List[int]]: Scores and indices of the top developers.
        """
        self._developer_prediction_model.eval()
        with torch.no_grad():
            tokenized_issue = self._developer_prediction_model.tokenize_text(issue)
            predictions = self._developer_prediction_model(tokenized_issue)
        output = torch.sum(torch.stack(predictions), 0)
        prediction_score, predicted_devs = output.topk(k, 1, True, True)
        return (
            F.softmax(prediction_score.squeeze(dim=0), dim=0).cpu().numpy().tolist(),
            predicted_devs.squeeze(dim=0).cpu().numpy().tolist()
        )

    def _get_predicted_components(self, issue: str, k: int) -> Tuple[List[float], List[int]]:
        """
        Generates component predictions using the component prediction model.

        Args:
            issue (str): The issue description.
            k (int): Number of top components to predict.

        Returns:
            Tuple[List[float], List[int]]: Scores and indices of the top components.
        """
        self._component_prediction_model.eval()
        with torch.no_grad():
            tokenized_issue = self._component_prediction_model.tokenize_text(issue)
            predictions = self._component_prediction_model(tokenized_issue)
        output = torch.sum(torch.stack(predictions), 0)
        prediction_score, predicted_components = output.topk(k, 1, True, True)
        return (
            F.softmax(prediction_score.squeeze(dim=0), dim=0).cpu().numpy().tolist(),
            predicted_components.squeeze(dim=0).cpu().numpy().tolist()
        )

    def _get_recommendation_by_similarity(
        self, issue: str, predicted_components: List[int], k_dev: int, k_issue: int, similarity_threshold: float
    ) -> List[Tuple[str, float]]:
        """
        Generates developer recommendations based on issue similarity.

        Args:
            issue (str): The issue description.
            predicted_components (List[int]): Predicted component IDs.
            k_dev (int): Number of top developers to recommend.
            k_issue (int): Number of top similar issues to consider.
            similarity_threshold (float): Similarity threshold to filter similar issues.

        Returns:
            List[Tuple[str, float]]: Developers and their similarity scores.
        """
        similar_issues = self._get_top_k_similar_issues(issue, k_issue, similarity_threshold)
        historical_contribution = self._get_historical_contributors(similar_issues, predicted_components)
        logger.debug(historical_contribution)
        return historical_contribution[:k_dev]
    
    def _get_historical_contributors(
        self, similar_issues: List[Tuple[int, float]], predicted_component_ids: List[int]
    ) -> List[Tuple[str, float]]:
        """
        Retrieves historical contributors from similar issues and ranks them.

        Args:
            similar_issues (List[Tuple[int, float]]): List of similar issue indices and similarity scores.
            predicted_component_ids (List[int]): Predicted component IDs.

        Returns:
            List[Tuple[str, float]]: Ranked developers and their contribution scores.
        """
        user_contribution_counts = {}
        skipped_users = set()
        for issue_index, sim_score in similar_issues:
            base_points = 1
            issue = self._train_data.iloc[issue_index]
            if self._component2id_map.get(issue.component) not in predicted_component_ids:
                logger.warning(
                    f"Skipping issue as label id {self._component2id_map.get(issue.component)} did not match with any of {predicted_component_ids}"
                )
                continue
            issue_number = issue.issue_number
            contributors = self._get_contribution_data(issue_number)
            for key, users in contributors.items():
                for user_data in users:
                    user = user_data[0]
                    created_at = user_data[1] if len(user_data) > 1 else None
                    
                    if user not in self._expected_developers:
                        skipped_users.add(user)
                        continue

                    contribution_point = 1
                    if key in ["pull_request", "commits"]:
                        contribution_point = 1.5
                    elif key in ["last_assignment", "direct_assignment"]:
                        contribution_point = 2

                    # Calculate time decay only if created_at is available
                    time_decay = 1
                    if created_at:
                        contribution_date = datetime.strptime(created_at, "%Y-%m-%dT%H:%M:%SZ")
                        days_since_contribution = (datetime.now() - contribution_date).days
                        time_decay = 1 / (1 + days_since_contribution / 30)

                        print("Contribution date:", contribution_date)
                        print("Time decay:", time_decay)

                    if user in self._component_developers_map.get(issue.component, []):
                        user_contribution_counts[user] = (
                            user_contribution_counts.get(user, 0) + base_points * contribution_point * time_decay
                        )
                    else:
                        user_contribution_counts[user] = (
                            user_contribution_counts.get(user, 0) + base_points * contribution_point * time_decay
                        )
        if skipped_users:
            logger.warning(
                f"Skipped users: {skipped_users} because they don't exist in the expected developers list"
            )
        return sorted(user_contribution_counts.items(), key=lambda x: x[1], reverse=True)

    def _get_contribution_data(self, issue_number: int) -> Dict[str, List[Tuple[str, str]]]:
        """
        Retrieves contribution data for a given issue.

        Args:
            issue_number (int): Issue number.

        Returns:
            Dict[str, List[Tuple[str, str]]]: Contribution data categorized by type of contribution,
                                            including the contributor and the contribution date.
        """
        contributions = {}
        issue_file = f"{issue_number}.json"
        last_assignment = None
        if issue_file in self._all_issues:
            with open(os.path.join(self._issues_path, issue_file), "r") as file:
                issue = json.load(file)
                assignees = issue.get("assignees", [])
                assignee_logins = [(assignee["login"], None) for assignee in assignees] if assignees else []
                contributions["direct_assignment"] = assignee_logins
                timeline = issue.get("timeline_data", [])
                pull_requests, commits, discussion = [], [], []
                for timeline_event in timeline:
                    event = timeline_event.get("event")
                    created_at = timeline_event.get("created_at")
                    if event == "cross-referenced" and timeline_event["source"].get("issue", {}).get("pull_request"):
                        actor = timeline_event["actor"]["login"]
                        pull_requests.append((actor, created_at))
                        last_assignment = actor
                    elif event == "referenced" and timeline_event.get("commit_url"):
                        actor = timeline_event["actor"]["login"]
                        commits.append((actor, created_at))
                        last_assignment = actor
                    elif event == "commented":
                        actor = timeline_event["actor"]["login"]
                        discussion.append((actor, created_at))
                contributions["pull_request"] = pull_requests
                contributions["commits"] = commits
                contributions["discussion"] = discussion
                contributions["last_assignment"] = [(last_assignment, None)] if last_assignment else []
        return contributions

    def _get_top_k_similar_issues(self, issue: str, k: int, threshold: float) -> List[Tuple[int, float]]:
        """
        Retrieves the top-k similar issues to a given issue based on similarity threshold.

        Args:
            issue (str): The issue description.
            k (int): Number of top similar issues to retrieve.
            threshold (float): Similarity threshold to filter similar issues.

        Returns:
            List[Tuple[int, float]]: Indices and similarity scores of the top similar issues.
        """
        test_embed = self._similarity_model.encode(issue)
        cos_sim = util.cos_sim(test_embed, self._all_embeddings)
        topk_values, topk_indices = torch.topk(cos_sim, k=k)
        topk_values = topk_values.cpu().numpy()[0]
        topk_indices = topk_indices.cpu().numpy()[0]
        return [
            (idx, sim_score) for idx, sim_score in zip(topk_indices, topk_values) if sim_score >= threshold
        ]


In [34]:
trx = TriagerX(
    component_prediction_model=comp_model,
    developer_prediction_model=dev_model,
    similarity_model=similarity_model,
    issues_path="/home/mdafifal.mamun/notebooks/triagerX/data/openj9_issue_data_6_7_24",
    train_embeddings=train_embeddings_path,
    component_developers_map=components,
    developer_id_map=lbl2idx,
    component_id_map=comp_lbl2id,
    train_data=df_train,
    device="cuda"
)

2024-06-19 11:16:23.367 | DEBUG    | __main__:__init__:69 - Using device: cuda
2024-06-19 11:16:23.368 | DEBUG    | __main__:__init__:70 - Loading embeddings for existing issues...


In [35]:
def get_recommendation(test_idx):
    test_data = df_test.iloc[test_idx]

    print("Issue URL: ", test_data["issue_url"])
    print("Actual owner: ", test_data["owner"])
    print("Actual component: ", test_data["component"])

    return trx.get_recommendation(
        test_data.text,
        k_comp=3,
        k_dev=3,
        k_rank=5,
        similarity_threshold=0.75
    )

In [36]:
get_recommendation(85)

2024-06-19 11:16:27.536 | DEBUG    | __main__:get_recommendation:89 - Predicting components...
2024-06-19 11:16:27.586 | INFO     | __main__:get_recommendation:92 - Predicted components: ['comp:vm', 'comp:jvmti', 'comp:test']
2024-06-19 11:16:27.587 | DEBUG    | __main__:get_recommendation:94 - Generating developer recommendations...
2024-06-19 11:16:27.635 | INFO     | __main__:get_recommendation:97 - Recommended developers: ['babsingh', 'fengxue-IS', 'gacholio', 'JasonFengJ9', 'thallium']
2024-06-19 11:16:27.636 | DEBUG    | __main__:get_recommendation:99 - Generating recommendations by similarity...
2024-06-19 11:16:27.669 | WARNING  | __main__:_get_historical_contributors:263 - Skipped users: {'r30shah', 'bhavanisn'} because they don't exist in the expected developers list
2024-06-19 11:16:27.670 | DEBUG    | __main__:_get_recommendation_by_similarity:201 - [('babsingh', 14.035420524803586), ('gacholio', 2.631070216013878), ('fengxue-IS', 2.1509054325955734), ('JasonFengJ9', 0.1211

Issue URL:  https://github.com/eclipse-openj9/openj9/issues/19039
Actual owner:  babsingh
Actual component:  comp:vm
Contribution date: 2023-03-10 06:29:25
Time decay: 0.060362173038229376
Contribution date: 2023-03-07 22:19:06
Time decay: 0.06012024048096193
Contribution date: 2023-03-08 21:19:35
Time decay: 0.06024096385542168
Contribution date: 2023-03-08 21:21:48
Time decay: 0.06024096385542168
Contribution date: 2023-03-08 21:37:38
Time decay: 0.06024096385542168
Contribution date: 2023-03-09 18:29:12
Time decay: 0.060362173038229376
Contribution date: 2023-03-09 18:32:23
Time decay: 0.060362173038229376
Contribution date: 2023-03-09 18:41:12
Time decay: 0.060362173038229376
Contribution date: 2023-03-09 18:42:27
Time decay: 0.060362173038229376
Contribution date: 2023-03-09 19:16:30
Time decay: 0.060362173038229376
Contribution date: 2023-03-09 19:24:41
Time decay: 0.060362173038229376
Contribution date: 2023-03-09 23:11:16
Time decay: 0.060362173038229376
Contribution date: 2023

{'predicted_components': ['comp:vm', 'comp:jvmti', 'comp:test'],
 'comp_prediction_score': [0.9999921321868896,
  4.635095137928147e-06,
  3.2684231427992927e-06],
 'predicted_developers': ['babsingh', 'fengxue-IS', 'gacholio'],
 'dev_prediction_score': [0.9143640398979187,
  0.037339914590120316,
  0.03222620114684105,
  0.01550647709518671,
  0.0005633810651488602],
 'similar_devs': [('babsingh', 14.035420524803586),
  ('gacholio', 2.631070216013878),
  ('fengxue-IS', 2.1509054325955734)],
 'combined_ranking': ['babsingh', 'fengxue-IS', 'gacholio']}

In [37]:
recommendations = []

for i in range(len(df_test)):
    rec = get_recommendation(i)
    recommendations.append(rec)

2024-06-19 11:16:28.143 | DEBUG    | __main__:get_recommendation:89 - Predicting components...
2024-06-19 11:16:28.190 | INFO     | __main__:get_recommendation:92 - Predicted components: ['comp:vm', 'comp:test', 'comp:gc']
2024-06-19 11:16:28.191 | DEBUG    | __main__:get_recommendation:94 - Generating developer recommendations...
2024-06-19 11:16:28.237 | INFO     | __main__:get_recommendation:97 - Recommended developers: ['pshipton', 'gacholio', 'JasonFengJ9', 'llxia', 'babsingh']
2024-06-19 11:16:28.239 | DEBUG    | __main__:get_recommendation:99 - Generating recommendations by similarity...
2024-06-19 11:16:28.263 | DEBUG    | __main__:_get_recommendation_by_similarity:201 - []
2024-06-19 11:16:28.264 | INFO     | __main__:get_recommendation:104 - Recommended developers by issue similarity: []
2024-06-19 11:16:28.266 | DEBUG    | __main__:get_recommendation:106 - Aggregating rankings...
2024-06-19 11:16:28.267 | INFO     | __main__:get_recommendation:109 - Recommended developers by

Issue URL:  https://github.com/eclipse-openj9/openj9/issues/18134
Actual owner:  gacholio
Actual component:  comp:vm
Issue URL:  https://github.com/eclipse-openj9/openj9/issues/18143
Actual owner:  pshipton
Actual component:  comp:test


2024-06-19 11:16:28.363 | INFO     | __main__:get_recommendation:97 - Recommended developers: ['pshipton', 'JasonFengJ9', 'keithc-ca', 'babsingh', 'tajila']
2024-06-19 11:16:28.365 | DEBUG    | __main__:get_recommendation:99 - Generating recommendations by similarity...
2024-06-19 11:16:28.385 | DEBUG    | __main__:_get_recommendation_by_similarity:201 - [('pshipton', 2.8896103896103895), ('keithc-ca', 0.025974025974025976)]
2024-06-19 11:16:28.387 | INFO     | __main__:get_recommendation:104 - Recommended developers by issue similarity: ['pshipton', 'keithc-ca']
2024-06-19 11:16:28.388 | DEBUG    | __main__:get_recommendation:106 - Aggregating rankings...
2024-06-19 11:16:28.389 | INFO     | __main__:get_recommendation:109 - Recommended developers by ranking aggregation: ['pshipton', 'JasonFengJ9', 'keithc-ca']
2024-06-19 11:16:28.391 | DEBUG    | __main__:get_recommendation:89 - Predicting components...
2024-06-19 11:16:28.424 | INFO     | __main__:get_recommendation:92 - Predicted c

Contribution date: 2021-05-20 16:23:26
Time decay: 0.025974025974025976
Contribution date: 2021-05-20 19:05:25
Time decay: 0.025974025974025976
Contribution date: 2021-05-20 19:10:27
Time decay: 0.025974025974025976
Contribution date: 2021-05-20 20:33:48
Time decay: 0.025974025974025976
Contribution date: 2023-09-14 20:17:48
Time decay: 0.0974025974025974
Contribution date: 2021-05-20 15:39:52
Time decay: 0.025974025974025976
Contribution date: 2021-05-20 15:58:51
Time decay: 0.025974025974025976
Contribution date: 2021-05-20 16:11:12
Time decay: 0.025974025974025976
Contribution date: 2021-05-20 16:22:08
Time decay: 0.025974025974025976
Contribution date: 2021-05-20 16:23:17
Time decay: 0.025974025974025976
Contribution date: 2021-05-20 16:31:50
Time decay: 0.025974025974025976
Contribution date: 2021-05-20 16:33:50
Time decay: 0.025974025974025976
Contribution date: 2021-05-20 17:36:10
Time decay: 0.025974025974025976
Contribution date: 2021-05-20 19:04:13
Time decay: 0.0259740259740

2024-06-19 11:16:28.588 | WARNING  | __main__:_get_historical_contributors:263 - Skipped users: {'BradleyWood'} because they don't exist in the expected developers list
2024-06-19 11:16:28.589 | DEBUG    | __main__:_get_recommendation_by_similarity:201 - [('fengxue-IS', 10.525361336897511), ('LinHu2016', 2.1330383480825956), ('pshipton', 0.7289162240336369), ('dmitripivkine', 0.2662733529990167), ('tajila', 0.22220622513731092), ('gacholio', 0.17663716831533008), ('ChengJin01', 0.07017543859649122), ('JasonFengJ9', 0.06293706293706294), ('babsingh', 0.04958677685950413)]
2024-06-19 11:16:28.590 | INFO     | __main__:get_recommendation:104 - Recommended developers by issue similarity: ['fengxue-IS', 'LinHu2016', 'pshipton']
2024-06-19 11:16:28.592 | DEBUG    | __main__:get_recommendation:106 - Aggregating rankings...
2024-06-19 11:16:28.593 | INFO     | __main__:get_recommendation:109 - Recommended developers by ranking aggregation: ['babsingh', 'pshipton', 'llxia']
2024-06-19 11:16:28.

...

Issue URL:  https://github.com/eclipse-openj9/openj9/issues/19499
Actual owner:  keithc-ca
Actual component:  comp:crypto
Contribution date: 2023-08-25 13:55:37
Time decay: 0.09146341463414634
Contribution date: 2023-06-26 22:55:58
Time decay: 0.07731958762886598
Contribution date: 2023-06-27 02:15:03
Time decay: 0.07731958762886598
Contribution date: 2023-06-27 13:49:55
Time decay: 0.07751937984496124
Contribution date: 2023-08-15 13:19:33
Time decay: 0.08875739644970414
Contribution date: 2023-08-15 15:34:21
Time decay: 0.08875739644970414
Contribution date: 2023-08-15 19:06:05
Time decay: 0.08875739644970414
Contribution date: 2023-08-15 19:38:04
Time decay: 0.08875739644970414
Contribution date: 2023-09-08 19:27:29
Time decay: 0.09554140127388534
Contribution date: 2023-10-24 19:35:38
Time decay: 0.11194029850746269
Contribution date: 2023-10-27 20:11:51
Time decay: 0.11320754716981134
Contribution date: 2022-06-09 17:59:23
Time decay: 0.03896103896103896
Contribution date: 2022-09

2024-06-19 11:16:42.834 | DEBUG    | __main__:_get_recommendation_by_similarity:201 - []
2024-06-19 11:16:42.836 | INFO     | __main__:get_recommendation:104 - Recommended developers by issue similarity: []
2024-06-19 11:16:42.838 | DEBUG    | __main__:get_recommendation:106 - Aggregating rankings...
2024-06-19 11:16:42.839 | INFO     | __main__:get_recommendation:109 - Recommended developers by ranking aggregation: ['JasonFengJ9', 'keithc-ca', 'pshipton']
2024-06-19 11:16:42.841 | DEBUG    | __main__:get_recommendation:89 - Predicting components...
2024-06-19 11:16:42.873 | INFO     | __main__:get_recommendation:92 - Predicted components: ['comp:vm', 'comp:build', 'comp:gc']
2024-06-19 11:16:42.874 | DEBUG    | __main__:get_recommendation:94 - Generating developer recommendations...
2024-06-19 11:16:42.908 | INFO     | __main__:get_recommendation:97 - Recommended developers: ['keithc-ca', 'pshipton', 'hangshao0', 'fengxue-IS', 'gacholio']
2024-06-19 11:16:42.910 | DEBUG    | __main__:

Issue URL:  https://github.com/eclipse-openj9/openj9/issues/19543
Actual owner:  pshipton
Actual component:  comp:vm
Issue URL:  https://github.com/eclipse-openj9/openj9/issues/19583
Actual owner:  JasonFengJ9
Actual component:  comp:vm
Contribution date: 2021-11-15 19:24:19
Time decay: 0.03073770491803279
Contribution date: 2023-02-03 04:43:32
Time decay: 0.05639097744360902
Contribution date: 2023-02-03 04:43:09
Time decay: 0.05639097744360902
Contribution date: 2023-01-25 20:34:30
Time decay: 0.05555555555555555
Contribution date: 2023-01-25 22:33:09
Time decay: 0.05555555555555555
Contribution date: 2023-01-25 22:53:10
Time decay: 0.05555555555555555
Contribution date: 2023-01-25 22:54:36
Time decay: 0.05555555555555555
Contribution date: 2023-01-26 17:16:03
Time decay: 0.05565862708719852
Contribution date: 2023-01-27 19:17:23
Time decay: 0.055762081784386616
Contribution date: 2023-01-27 19:24:44
Time decay: 0.055762081784386616
Contribution date: 2023-01-27 19:24:55
Time decay: 

2024-06-19 11:16:43.061 | INFO     | __main__:get_recommendation:92 - Predicted components: ['comp:test', 'comp:build', 'comp:vm']
2024-06-19 11:16:43.062 | DEBUG    | __main__:get_recommendation:94 - Generating developer recommendations...
2024-06-19 11:16:43.098 | INFO     | __main__:get_recommendation:97 - Recommended developers: ['keithc-ca', 'pshipton', 'llxia', 'JasonFengJ9', 'babsingh']
2024-06-19 11:16:43.099 | DEBUG    | __main__:get_recommendation:99 - Generating recommendations by similarity...
2024-06-19 11:16:43.127 | WARNING  | __main__:_get_historical_contributors:263 - Skipped users: {'psoujany'} because they don't exist in the expected developers list
2024-06-19 11:16:43.129 | DEBUG    | __main__:_get_recommendation_by_similarity:201 - [('JasonFengJ9', 4.1964248159831765), ('pshipton', 2.327395871325508), ('LongyuZhang', 2.231447465099192), ('babsingh', 0.3745507921756673), ('ChengJin01', 0.3430710569679787), ('fengxue-IS', 0.07788212330582145), ('tajila', 0.0616332982

Contribution date: 2021-11-18 22:35:25
Time decay: 0.03083247687564234
Contribution date: 2021-11-24 17:17:01
Time decay: 0.031023784901758014
Contribution date: 2021-11-30 21:55:22
Time decay: 0.03121748178980229
Contribution date: 2021-11-30 22:06:38
Time decay: 0.03121748178980229
Contribution date: 2021-12-06 21:26:16
Time decay: 0.031413612565445025
Contribution date: 2021-12-10 17:34:13
Time decay: 0.031545741324921134
Contribution date: 2021-12-13 16:36:23
Time decay: 0.03164556962025316
Contribution date: 2021-12-14 16:58:52
Time decay: 0.03167898627243928
Contribution date: 2022-02-09 21:18:26
Time decay: 0.033707865168539325
Contribution date: 2021-11-30 21:54:39
Time decay: 0.03121748178980229
Contribution date: 2021-12-01 16:07:11
Time decay: 0.03125
Contribution date: 2021-11-17 20:02:04
Time decay: 0.030800821355236138
Contribution date: 2021-11-17 21:26:39
Time decay: 0.030800821355236138
Contribution date: 2021-11-18 17:38:07
Time decay: 0.03083247687564234
Contribution

In [38]:
combined_total = 0
dl_total = 0
sim_total = 0

for idx in range(len(df_test)):
    actual = df_test.iloc[idx]["owner"]
    combined_recommended = recommendations[idx]["combined_ranking"]
    dl_recommended = recommendations[idx]["predicted_developers"]
    sim_recommended = recommendations[idx]["similar_devs"]

    if sim_recommended:
        sim_recommended = [s[0] for s in sim_recommended]

    print(idx, end=" ")

    if actual in combined_recommended:
        print("Comb,", end="")
        combined_total += 1

    if actual in dl_recommended:
        print("DL,", end="")
        dl_total += 1

    if actual in sim_recommended:
        print("Sim", end="")
        sim_total += 1

    print("===")

0 Comb,DL,===
1 Comb,DL,Sim===
2 Comb,DL,===
3 Comb,DL,===
4 ===
5 Comb,DL,Sim===
6 Comb,DL,Sim===
7 Comb,DL,Sim===
8 ===
9 ===
10 ===
11 Comb,DL,Sim===
12 ===
13 Comb,DL,===
14 Comb,DL,Sim===
15 Comb,DL,===
16 Comb,DL,===
17 Comb,DL,Sim===
18 ===
19 ===
20 Comb,DL,Sim===
21 Comb,DL,===
22 Comb,DL,===
23 ===
24 Comb,DL,===
25 Comb,DL,Sim===
26 Comb,DL,Sim===
27 Comb,Sim===
28 Sim===
29 Comb,DL,Sim===
30 Comb,DL,===
31 Comb,DL,Sim===
32 ===
33 Comb,DL,Sim===
34 Comb,DL,===
35 Comb,DL,===
36 Comb,Sim===
37 ===
38 ===
39 Comb,DL,Sim===
40 Comb,DL,===
41 ===
42 Sim===
43 ===
44 ===
45 Comb,DL,===
46 ===
47 ===
48 Comb,DL,===
49 Comb,DL,Sim===
50 Comb,DL,===
51 ===
52 ===
53 Comb,DL,Sim===
54 ===
55 Comb,DL,===
56 Comb,DL,===
57 ===
58 ===
59 Comb,DL,Sim===
60 Comb,DL,Sim===
61 Comb,DL,Sim===
62 ===
63 Comb,DL,Sim===
64 Comb,DL,===
65 Comb,DL,Sim===
66 Comb,DL,===
67 Comb,DL,===
68 Comb,DL,Sim===
69 Comb,DL,Sim===
70 Comb,DL,Sim===
71 Comb,DL,Sim===
72 Comb,DL,===
73 Sim===
74 Comb,DL,Sim==

In [39]:
sim_total

55

In [40]:
dl_total/len(df_test), sim_total/len(df_test), combined_total/len(df_test)

(0.6666666666666666, 0.4074074074074074, 0.7185185185185186)

In [32]:
anal_id = 47

print(df_test.iloc[anal_id]["issue_url"])
df_test.iloc[anal_id], recommendations[anal_id]

https://github.com/eclipse-openj9/openj9/issues/18718


(issue_number                                                18718
 issue_title     JDK22 MHProxies.asInterfaceInstance: named mod...
 description     ### Source Code\r\n\r\n`MethodHandleProxies.as...
 issue_url       https://github.com/eclipse-openj9/openj9/issue...
 issue_state                                                closed
 creator                                                  babsingh
 labels                               comp:vm, test failure, jdk22
 owner                                                    babsingh
 component                                                 comp:vm
 text            Bug Title: JDK22 MHProxies.asInterfaceInstance...
 owner_id                                                        5
 Name: 7562, dtype: object,
 {'predicted_components': ['comp:vm', 'comp:build', 'comp:jclextensions'],
  'comp_prediction_score': [1.0,
   7.691897119244118e-10,
   1.1702369628885378e-10],
  'predicted_developers': ['JasonFengJ9', 'keithc-ca', 'hangshao0'],
  'd